# Dependency

In [ ]:
from dependency import *

# Simulate random uniform instance

In [ ]:
def simulate_instance(n = 20,
                      m = 6,
                      min_vehicle_arrival = 1,
                      max_vehicle_arrival = 100,
                      min_waiting_time = 1,
                      max_waiting_time = 6,
                      min_chargers_availability_time = 1,
                      max_chargers_availability_time = 20,
                      min_required_energy = 30,
                      max_required_energy = 100,
                      charging_powers = [10,20,30,40],
                      std_deviations_values = np.linspace(0.05,0.2,10),
                      min_perc_compatibility = 50,
                      kappa = 1):

    A = np.zeros((n,m), dtype = np.int8)
    requests = np.zeros((n,3))
    for i in range(n):
        arrival = np.random.uniform(min_vehicle_arrival, max_vehicle_arrival)

        waiting_time = np.random.uniform(min_waiting_time, max_waiting_time)

        departure = np.random.uniform(arrival, arrival+waiting_time)
        
        energy = np.random.uniform(min_required_energy, max_required_energy)

        requests[i] = arrival, departure, energy

        num_compatible_chargers = np.random.randint(ceil((min_perc_compatibility/100) * m), m+1)
        
        compatible_chargers = np.random.choice(np.arange(m), num_compatible_chargers, replace = False)
        
        A[i, compatible_chargers] = 1

    chargers = np.zeros((m,2))
    for i in range(m):
        availability_time = np.random.uniform(min_chargers_availability_time, max_chargers_availability_time)
        power = np.random.choice(charging_powers)
        chargers[i] = availability_time, power

    std_deviations = np.zeros((n,m))
    for i in range(n):
        for j in range(m):
            if A[i,j] == 1:
                std_deviations[i,j] = np.random.choice(std_deviations_values)

    Instance = (requests, chargers, A, kappa, std_deviations)
    return Instance

Instance = simulate_instance()
requests, chargers, A, kappa, std_deviations = Instance

# Generate and plot a random schedule

In [ ]:
Instance = simulate_instance(n = 50, m = 20)

requests, chargers, A, kappa, std_deviations = Instance

sigma, min_percentage, min_distance_between_vehicles, min_pl, min_npl, max_preemptions = 5, 20, 0.1, 0.1, 0.1, 5

schedule = generate_random_schedule(Instance, sigma, min_percentage, min_distance_between_vehicles, min_pl, min_npl, max_preemptions)
    
plot_schedule(Instance, schedule, figsize=(20, 13))

# NSGA-II

## Test

In [ ]:
Instance = simulate_instance(n = 30, m = 10)

requests, chargers, A, kappa, std_deviations = Instance

sigma, min_percentage, min_distance_between_vehicles, min_pl, min_npl, max_preemptions = 5, 50, 0.1, 0.1, 0.1, 20

num_replications, p_m = 20, 0.1

other_args = sigma, min_percentage, min_distance_between_vehicles, min_pl, min_npl, max_preemptions, num_replications, p_m

result = NSGA2(Instance, other_args, inverse_signs = np.array([1,1,1]), population_size = 100, generations = 100, mutation_rate=0.2, p_sel = 0.3, show_progress = True)

# NSGA-III

## Test

In [ ]:
ref_points = generate_reference_points(num_objectives = 3, divisions=20)

Instance = simulate_instance(n = 20, m = 10)

requests, chargers, A, kappa, std_deviations = Instance

sigma, min_percentage, min_distance_between_vehicles, min_pl, min_npl, max_preemptions = 5, 50, 0.1, 0.1, 0.1, 2

num_replications, p_m = 30, 0.1

other_args = sigma, min_percentage, min_distance_between_vehicles, min_pl, min_npl, max_preemptions, num_replications, p_m, ref_points

result = NSGA3(Instance, other_args, inverse_signs = np.array([1,1,1]), population_size = 100, generations = 100, mutation_rate=0.2, p_sel = 0.3, show_progress = True)

# MOCS

## Test

In [ ]:
Instance = simulate_instance(n = 50, m = 10)

requests, chargers, A, kappa, std_deviations = Instance

sigma, min_percentage, min_distance_between_vehicles, min_pl, min_npl, max_preemptions = 5, 50, 0.1, 0.1, 0.1, 2

num_replications, pc = 30, 0.1

other_args = sigma, min_percentage, min_distance_between_vehicles, min_pl, min_npl, max_preemptions, num_replications, pc

result = MOCS(Instance, other_args, inverse_signs = np.array([1,1,1]), population_size = 100, generations = 150, perc_abandoned = 0.2, p_sel = 0.3, show_progress = True)

# MOGWO

## Test

In [ ]:
Instance = simulate_instance(n = 20, m = 10)

requests, chargers, A, kappa, std_deviations = Instance

num_replications, sigma, min_percentage, min_distance_between_vehicles, min_pl, min_npl, max_preemptions = 30, 5, 50, 0.1, 0.1, 0.1, 2

archive_maxsize, num_points_per_dim, alpha, beta = 1000, 10, 0.5, 0.5

mutation_rate, p_m = 0.1, 0.1

other_args = archive_maxsize, num_points_per_dim, alpha, beta, num_replications, sigma, min_percentage, min_distance_between_vehicles, min_pl, min_npl, max_preemptions, mutation_rate, p_m

result = MOGWO(Instance, other_args, inverse_signs = np.array([1,1,1]), population_size = 100, generations = 150, show_progress = True)